# Momentos Invariantes de Hu

In [44]:
# imports
import cv2
from pathlib import Path
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [45]:
def hu_moments(image, quadrants=(1,1)):
    # Tamanho dos quadrantes
    width, height = image.shape[0], image.shape[1]
    part_width = width // quadrants[0]
    part_height = height // quadrants[1]
    
    # Calculate
    huMoments = list()
    for i in range(quadrants[0]):
        for j in range(quadrants[1]):
            #coordenadas da parte da imagem
            left = i * part_width
            upper = j * part_height
            right = left + part_width
            lower = upper + part_height

            # Retorna o quadrante da imagem
            quad = image[left:right, upper:lower]

            # Calculate Moments 
            moments = cv2.moments(quad)
            # Calculate Hu Moments
            huMoments.extend(cv2.HuMoments(moments).flatten())
            
    return huMoments


In [46]:
def black_pixels(image, quadrants=(1,1)):
    # Tamanho dos quadrantes
    width, height = image.shape[0], image.shape[1]
    part_width = width // quadrants[0]
    part_height = height // quadrants[1]

    counts = []
    for i in range(quadrants[0]):
        for j in range(quadrants[1]):
            #coordenadas da parte da imagem
            left = i * part_width
            upper = j * part_height
            right = left + part_width
            lower = upper + part_height

            # Retorna o quadrante da imagem
            part = image[left:right, upper:lower]

            # Contar os pixels pretos (assumindo que menores que 128 são pretos)
            black_pixels = np.sum(np.array(part) < 128)

            # Adicionar a contagem à lista
            counts.append(black_pixels)

    return counts

In [47]:
def get_files(caminho_p):
    file_dict = {}
    for pasta in Path(caminho_p).iterdir():
        if pasta.is_dir():
            file_dict[pasta.name] = []
            
            for file in Path(pasta).iterdir():
                if file.is_file() and (file.name.endswith(".png") or file.name.endswith(".jpg")):
                    file_dict[pasta.name].append(file)
    return file_dict

### Create X, y

In [48]:
database = get_files('../fourShapes/')

quadrantes = (2,2)
X_hu, X_bp, y = list(), list(), list()

for classe in database:
    for item in database[classe]:        
        image = cv2.imread(str(item), cv2.IMREAD_GRAYSCALE) #open img

        #hu
        X_hu.append(hu_moments(image, quadrantes))
        X_bp.append(black_pixels(image, quadrantes))
        y.append(classe)

X_hu, X_bp, y = np.array(X_hu), np.array(X_bp), np.array(y)
print(X_hu.shape, X_bp.shape, y.shape)

(4000, 175) (4000, 25) (4000,)


### k-NN

In [49]:
# normalize
X_hu = StandardScaler().fit_transform(X_hu)
X_bp = StandardScaler().fit_transform(X_bp)

# separa em conjunto de treino e teste
X_hu_train, X_hu_test, y_hu_train, y_hu_test = train_test_split(X_hu, y, test_size=0.4)
X_bp_train, X_bp_test, y_bp_train, y_bp_test = train_test_split(X_bp, y, test_size=0.4)

print('HU: Conjunto de treino (X, y) e teste (X, y):', X_hu_train.shape, y_hu_train.shape, X_hu_test.shape, y_hu_test.shape)
print('BP: Conjunto de treino (X, y) e teste (X, y):', X_bp_train.shape, y_bp_train.shape, X_bp_test.shape, y_bp_test.shape)

HU: Conjunto de treino (X, y) e teste (X, y): (2400, 175) (2400,) (1600, 175) (1600,)
BP: Conjunto de treino (X, y) e teste (X, y): (2400, 25) (2400,) (1600, 25) (1600,)


In [50]:
#knn
neighbors = 3

#momentos de hu
knn_hu = KNeighborsClassifier(n_neighbors=neighbors, metric='euclidean')
knn_hu.fit(X_hu_train, y_hu_train)
y_hu_pred = knn_hu.predict(X_hu_test)

print('Hu Moments:')
print('accuracy:', accuracy_score(y_hu_test, y_hu_pred))
# print(confusion_matrix(y_hu_test, y_hu_pred))
# print(classification_report(y_hu_test, y_hu_pred, labels=knn_hu.classes_))
print()

#black pixels
knn_bp = KNeighborsClassifier(n_neighbors=neighbors, metric='euclidean')
knn_bp.fit(X_bp_train, y_bp_train)
y_bp_pred = knn_bp.predict(X_bp_test)

print('Black Pixels:')
print('accuracy:', accuracy_score(y_bp_test, y_bp_pred))
# print(confusion_matrix(y_bp_test, y_bp_pred))
# print(classification_report(y_bp_test, y_bp_pred, labels=knn_bp.classes_))

Hu Moments:
accuracy: 0.958125

Black Pixels:
accuracy: 0.993125
